### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [2]:
import pandas as pd
from collections import Counter
from nltk.stem import SnowballStemmer 
russian_stemmer = SnowballStemmer('russian')
print()

### Вывод

Из информации о данных, можно заметить, что days_employed (стаж в днях) должен иметь тип данных int а не float, а также, что <b>days_employed</b> и <b>total_income</b> содержат пропуски, их число одинаковое, поэтому возможно они как-то связаны. С помощью метода head() мы узнали, что <b> days_employed </b> содержат отрицательные значения, чего конечно же быть не должно, <b> education </b> - в разных регистрах. В общем, данные совсем не идеальные. Пробелов в названии столбцов нет - хоть что-то хорошее.



### Шаг 2. Предобработка данных

### Обработка пропусков

In [899]:
#Посчитаем пропуски
data['days_employed'].isnull().sum()

2174

In [3]:
data_emply = data[data['days_employed'].isnull() & data['total_income'].isnull()]
data_emply.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Мы выяснили, что, действительно, там где пропущено значение <b> 'days_employed' </b> пропущено и значение <b> 'total_income'</b> , можно было бы предположить, что это ещё учащиеся люди, которые никогда не работали, но у них совершенно разный возраст, и у некоторых есть дети. Наверное, какая - то техническая ошибка.

In [901]:
#Заполним пропуски в days_employed нулями, 
# а в total_income медианным значением, для людей с таким же образованием, типом занятости и семейным положением
data['days_employed'] = data['days_employed'].fillna(0) 
data['total_income'] = data['total_income'].fillna(0) 

In [4]:
data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.median()))
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

При просмотре информации выяснилось. что в датасете 2174 пропуск в 'days_employed' и в 'total_income'. Мы выяснили, что эти пропуски взаимосвязаны, там где есть один, есть и другой. Мы заполнили Nan-ы в days_employed нулями, а в total_income - медианным значением, среди людей, с таким же образованием, семейным положением и типом занятости.

<h3>Найдём и исправим некоторые другие проблемы данных: </h3>

Начнём с регистров, просмотрим столбцы, где могут встретиться подобные ошибки:

In [903]:
#Проверим, только ли в 'education' проблемы с регистром
print(data['family_status'].unique())
print(data['education'].unique())
print(data['income_type'].unique())

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


Проблемы есть только в education, переведём его в нижний регистр и проверим результат:

In [904]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [905]:
data.min()

children                               -1
days_employed                    -18388.9
dob_years                               0
education                          высшее
education_id                            0
family_status       Не женат / не замужем
family_status_id                        0
gender                                  F
income_type                   безработный
debt                                    0
total_income                            0
purpose                        автомобили
dtype: object

Посмотрим на отрицательных детей и отрицательные зарплаты, и исправим их на их модули, также посмотрим на людей, которым 0 лет и удалим их из таблицы, 

In [906]:
problem_negative_children = data[ data['children'] < 0]
print('строк с отрицательными детьми -',problem_negative_children['children'].count())
data['children'] = abs(data['children'])
problem_negative_employed = data[ data['days_employed'] < 0]
print('строк с отрицательным стажем -',problem_negative_employed['days_employed'].count())
data['days_employed'] = abs(data['days_employed'])
low_years = data[data['dob_years'] ==0]
print('строк со значением возраста равным 0 -',low_years['dob_years'].count())
data = data[data['dob_years'] > 0]
#Проверим результат
data.min()

строк с отрицательными детьми - 47
строк с отрицательным стажем - 15906
строк со значением возраста равным 0 - 101


children                                0
days_employed                           0
dob_years                              19
education                          высшее
education_id                            0
family_status       Не женат / не замужем
family_status_id                        0
gender                                  F
income_type                   безработный
debt                                    0
total_income                            0
purpose                        автомобили
dtype: object

In [907]:
data.max()

children                         20
days_employed                401755
dob_years                        75
education            ученая степень
education_id                      4
family_status       женат / замужем
family_status_id                  4
gender                          XNA
income_type                 студент
debt                              1
total_income             2.2656e+06
purpose             сыграть свадьбу
dtype: object

Заметим, подозрительные строки: 20 детей, 1100 лет рабочего стажа и гендер - XNA (я загуглил, нет такого гендера). Проверим их

In [908]:
print(Counter(data['gender']))
data = data[data['gender'] !='XNA']
#человек с таким гендером оказался 1, поскольку имени или другой идентифицирующей информации у нас нет удалим его из базы
data.info()

Counter({'F': 14164, 'M': 7259, 'XNA': 1})
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21423 entries, 0 to 21524
Data columns (total 12 columns):
children            21423 non-null int64
days_employed       21423 non-null float64
dob_years           21423 non-null int64
education           21423 non-null object
education_id        21423 non-null int64
family_status       21423 non-null object
family_status_id    21423 non-null int64
gender              21423 non-null object
income_type         21423 non-null object
debt                21423 non-null int64
total_income        21423 non-null float64
purpose             21423 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


In [5]:
#Узнаем сколько всего человек, у которых стаж больше возраста и посмотрим на них
data[data['days_employed']/365 > data['dob_years']].info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3445 entries, 4 to 21521
Data columns (total 12 columns):
children            3445 non-null int64
days_employed       3445 non-null float64
dob_years           3445 non-null int64
education           3445 non-null object
education_id        3445 non-null int64
family_status       3445 non-null object
family_status_id    3445 non-null int64
gender              3445 non-null object
income_type         3445 non-null object
debt                3445 non-null int64
total_income        3445 non-null float64
purpose             3445 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 349.9+ KB


Как видно, все пенсионеры, выясним, так ли это:

In [910]:
#retirees - пенсионеры
retirees = data[data['income_type'] == 'пенсионер'] 
print('Всего пенсионеров:',retirees['income_type'].count()) #3836
retirees_with_problem1 = retirees[ retirees['days_employed']/365 >= retirees['dob_years']] #3426
print('Пенсионеров, у которых стаж больше возраста:',retirees_with_problem1['income_type'].count()) 
retirees_with_problem2 = retirees [retirees['days_employed'] == 0] 
print('Пенсионеров, у которых стаж равен нулю:',retirees_with_problem2['income_type'].count()) 

Всего пенсионеров: 3836
Пенсионеров, у которых стаж больше возраста: 3426
Пенсионеров, у которых стаж равен нулю: 410


Получилось, что из 3836 пенсионеров у 3446 стаж работы больше чем возраст, а у  410 стаж равен нулю. Никуда конечно не привели нас эти размышления, но зависимость интересная, ни одной строчки без ошибки у пенсионеров.

C 20-ю детьми я думаю возникла опечатка, заменим 20 на 2 и проверим результат:

In [911]:
print('До:',Counter(data['children']))
data['children'] = data['children'].replace(20,2)
print('После:',Counter(data['children']))

До: Counter({0: 14079, 1: 4849, 2: 2042, 3: 328, 20: 75, 4: 41, 5: 9})
После: Counter({0: 14079, 1: 4849, 2: 2117, 3: 328, 4: 41, 5: 9})


### Изменение типа данных

Значения столбцов зарплаты и стажа трудовых дней заменим на целочисленные и проверим:

In [912]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21423 entries, 0 to 21524
Data columns (total 12 columns):
children            21423 non-null int64
days_employed       21423 non-null int64
dob_years           21423 non-null int64
education           21423 non-null object
education_id        21423 non-null int64
family_status       21423 non-null object
family_status_id    21423 non-null int64
gender              21423 non-null object
income_type         21423 non-null object
debt                21423 non-null int64
total_income        21423 non-null int64
purpose             21423 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод

Как было сказано в начале, столбец <b>'days_employed'</b> - стаж в днях, должен быть целочисленным, мы привели его к целочисленному виду методом astype() с аргументом 'int', потому что это единственный изученный метод, позволяющий преобразовывать вещественный тип в целочисленный. Методом info() проверили, что всё прошло удачно. 

### Обработка дубликатов

In [913]:
#Посмотрим на таблицу дубликатов, чтобы выявить общие тенденции и понять с чем может быть связано их образование:
duplicates = data[data.duplicated()]
print(duplicates.head(5))
#посчитаем кол-во дубликатов методом duplicated() с методом sum() и выведем его:
print('Кол-во дубликатов:' ,data.duplicated().sum())
#Посчитаем дубликаты с ошибкой в days_employed
dub_null = duplicates[duplicates['days_employed'] == 0]['days_employed'].count()
print('Дубликатов, с нарущенным days_employed :',dub_null)
#Удалим дубликаты методом drop_duplicates() с методом перерасчёта индексов после удаления reset_index(drop = True)
data = data.drop_duplicates().reset_index(drop = True)
print('Проверка: дубликатов после удаления:' ,data.duplicated().sum())

      children  days_employed  dob_years education  education_id  \
2849         0              0         41   среднее             1   
3290         0              0         58   среднее             1   
4182         1              0         34    высшее             0   
4851         0              0         60   среднее             1   
5557         0              0         58   среднее             1   

         family_status  family_status_id gender income_type  debt  \
2849   женат / замужем                 0      F   сотрудник     0   
3290  гражданский брак                 1      F   пенсионер     0   
4182  гражданский брак                 1      F   сотрудник     0   
4851  гражданский брак                 1      F   пенсионер     0   
5557  гражданский брак                 1      F   пенсионер     0   

      total_income                  purpose  
2849        128132  покупка жилья для семьи  
3290        104903          сыграть свадьбу  
4182        150897                  св

### Вывод

Дубликатов оказалось не очень много, всего 71, но у всех у них изначально была ошибка в столбце total_income, обычно дубликаты появляются из-за технических ошибок.


### Лемматизация

In [914]:
# Импортруем всё что нужно для успешной лемматизации
from pymystem3 import Mystem
m = Mystem()
# Напишем функцию, для лемматизации
def create_good_lemma(row):
    purpose = row['purpose']
    lemma = ''.join(m.lemmatize(purpose))
    lemma = lemma.replace("\n","")
    return lemma
# Применим функцию и посмотрим на результат, а также на самые популярные цели кредита
data['lem_purpose'] = data.apply (create_good_lemma, axis =1)
print(data.head())
Counter(data['lem_purpose']).most_common

   children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   среднее             1   
3         3           4124         32   среднее             1   
4         0         340266         53   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M   сотрудник     0        145885   
3   женат / замужем                 0      M   сотрудник     0        267628   
4  гражданский брак                 1      F   пенсионер     0        158616   

                      purpose                 lem_purpose  
0               покупка жилья               покупка жилье  
1     пр

<bound method Counter.most_common of Counter({'автомобиль': 966, 'свадьба': 786, 'на проведение свадьба': 764, 'сыграть свадьба': 760, 'операция с недвижимость': 672, 'покупка коммерческий недвижимость': 658, 'покупка жилье для сдача': 649, 'операция с коммерческий недвижимость': 648, 'операция с жилье': 646, 'покупка жилье': 640, 'жилье': 640, 'покупка жилье для семья': 637, 'строительство собственный недвижимость': 633, 'недвижимость': 629, 'операция со свой недвижимость': 627, 'строительство жилой недвижимость': 621, 'строительство недвижимость': 619, 'покупка свой жилье': 619, 'покупка недвижимость': 617, 'ремонт жилье': 605, 'покупка жилой недвижимость': 603, 'на покупка свой автомобиль': 502, 'заниматься высокий образование': 493, 'сделка с подержанный автомобиль': 484, 'на покупка подержать автомобиль': 478, 'свой автомобиль': 475, 'на покупка автомобиль': 466, 'приобретение автомобиль': 459, 'дополнительный образование': 458, 'сделка с автомобиль': 454, 'высокий образование': 4

### Вывод

Мы провели лемматизацию с помощью пакета pymystem3, создав функцию, которая приводит слова в начальную форму и удаляет символ '\n', применив эту функцию построчно и добавив в новый столбец lem_purpose леммы, мы посмотрели на самые популярные цели кредита, ими оказались: покупка автомобиля, проведение свадьбы, покупка жилья и образование.

### Категоризация данных

In [915]:
# Сделаемc сводную таблицу по количеству детей и возврату долга
children_pivot_table = data.groupby(['debt', 'children'])['gender'].aggregate('count').unstack()
# Сделаемc сводную таблицу по семейному статусу и возрату долга
fam_stat_pivot_tab = data.groupby(['debt', 'family_status'])['gender'].aggregate('count').unstack()

In [916]:
# Категоризируем по уровню заработка
MROT = 12130
average_income = 44076
def income_group(income):
    if income < MROT:
        return 'Доход ниже МРОТа'
    elif income > 10 * average_income:
        return 'Доход очень сильно выше среднего'
    elif income > 5 * average_income:
        return 'Доход сильно выше среднего'
    elif income > average_income:
        return 'Доход выше среднего'
    else:
        return 'Доход ниже среднего'
data['income_group'] = data['total_income'].apply(income_group)
data['income_group'].value_counts()

income_pivot_table = data.groupby(['debt', 'income_group'])['gender'].aggregate('count').unstack()
income_pivot_table

income_group,Доход выше среднего,Доход ниже МРОТа,Доход ниже среднего,Доход очень сильно выше среднего,Доход сильно выше среднего
debt,,,,,
0,15737.0,2.0,186.0,339.0,3355.0
1,1439.0,NaN,13.0,22.0,259.0


### Вывод

Мы сделали 2 сводные таблицы отражающие зависимость мужду наличием детей и возвратом кредита в срок и зависимость между семейным статусом и возвратом кредита в срок. Также разделили всех заёмщиков на категории по доходу, и сделали для них сводную таблицу, выражающую зависимость между доходом и возвратом кредита в срок. Ни одного человека с доходом ниже МРОТа не обнаружено.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [917]:
# Распечатаем сводную таблицу
print(children_pivot_table)
# Посчитаем процент возврата для каждого случая
for_0_children = children_pivot_table[0][0] / children_pivot_table[0].sum() 
for_1_children = children_pivot_table[1][0] / children_pivot_table[1].sum() 
for_2_children = children_pivot_table[2][0] / children_pivot_table[2].sum() 
for_3_children = children_pivot_table[3][0] / children_pivot_table[3].sum() 
for_4_children = children_pivot_table[4][0] / children_pivot_table[4].sum() 
for_5_children = children_pivot_table[5][0] / children_pivot_table[5].sum() 
# Выведем результат
print ()
print('Процент людей, не имеющих задолжности:')
print('Для людей без детей: {:.3f}'.format(for_0_children))
print('Для людей с 1 ребёнком: {:.3f}'.format(for_1_children))
print('Для людей с 2 детьми: {:.3f}'.format(for_2_children))
print('Для людей с 3 детьми: {:.3f}'.format(for_3_children))
print('Для людей с 4 детьми: {:.3f}'.format(for_4_children))
print('Для людей с 5 детьми: {:.3f}'.format(for_5_children))

children        0       1       2      3     4    5
debt                                               
0         12963.0  4397.0  1912.0  301.0  37.0  9.0
1          1058.0   442.0   202.0   27.0   4.0  NaN

Процент людей, не имеющих задолжности:
Для людей без детей: 0.925
Для людей с 1 ребёнком: 0.909
Для людей с 2 детьми: 0.904
Для людей с 3 детьми: 0.918
Для людей с 4 детьми: 0.902
Для людей с 5 детьми: 1.000


### Вывод

Семьи без детей чаще возвращают долги, чем семьи с детьми. Семьи с 3я детьми возвращают долги чаще, чем семьи с 1,2 или 4-мя детьми. Для семей с 5-ю детьми вероятность погашения - 100%, но там слишком узкая выборка (9 человек) и судить по ней думаю некорректно, но результат получился контринтуитивным.


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [918]:
# Распечатаем сводную таблицу:
print(fam_stat_pivot_tab)
# Посчитаем процент возврата для каждого случая:
for_0_fam_stat = fam_stat_pivot_tab['Не женат / не замужем'][0] / fam_stat_pivot_tab['Не женат / не замужем'].sum() 
for_1_fam_stat = fam_stat_pivot_tab['в разводе'][0] / fam_stat_pivot_tab['в разводе'].sum() 
for_2_fam_stat = fam_stat_pivot_tab['вдовец / вдова'][0] / fam_stat_pivot_tab['вдовец / вдова'].sum() 
for_3_fam_stat = fam_stat_pivot_tab['гражданский брак'][0] / fam_stat_pivot_tab['гражданский брак'].sum() 
for_4_fam_stat = fam_stat_pivot_tab['женат / замужем'][0] / fam_stat_pivot_tab['женат / замужем'].sum() 
# Выведем результат:
print ()
print('Процент людей, не имеющих задолжности:')
print('Для неженатых людей: {:.3f}'.format(for_0_fam_stat))
print('Для людей в разводе: {:.3f}'.format(for_1_fam_stat))
print('Для вдов и вдовцов: {:.3f}'.format(for_2_fam_stat))
print('Для людей, состоящих в гражданском браке: {:.3f}'.format(for_3_fam_stat))
print('Для женатых людей: {:.3f}'.format(for_4_fam_stat))

family_status  Не женат / не замужем  в разводе  вдовец / вдова  \
debt                                                              
0                               2521       1100             892   
1                                273         85              62   

family_status  гражданский брак  женат / замужем  
debt                                              
0                          3743            11363  
1                           386              927  

Процент людей, не имеющих задолжности:
Для неженатых людей: 0.902
Для людей в разводе: 0.928
Для вдов и вдовцов: 0.935
Для людей, состоящих в гражданском браке: 0.907
Для женатых людей: 0.925


### Вывод

Вдовцы и вдовы чаще остальных возвращают долги, а неженатые - делают это реже всех. Люди в разводе возвращают почти также часто, как и женатые люди, а люди в гражданском браке почти также редко, как и неженатые.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [919]:
print(income_pivot_table)
for_1_income = income_pivot_table['Доход выше среднего'][0] / income_pivot_table['Доход выше среднего'].sum() 
for_2_income = income_pivot_table['Доход ниже среднего'][0] / income_pivot_table['Доход ниже среднего'].sum() 
for_3_income = income_pivot_table['Доход очень сильно выше среднего'][0] / income_pivot_table['Доход очень сильно выше среднего'].sum() 
for_4_income = income_pivot_table['Доход сильно выше среднего'][0] / income_pivot_table['Доход сильно выше среднего'].sum() 
print ()
print('Процент людей, не имеющих задолжности:')
print('Для людей, чей доход выше среднего: {:.3f}'.format(for_1_income))
print('Для людей, чей доход ниже среднего: {:.3f}'.format(for_2_income))
print('Для людей, чей доход очень сильно выше среднего (в 10 и более раз): {:.3f}'.format(for_3_income))
print('Для людей, чей доход выше среднего (в 5-10 раз): {:.3f}'.format(for_4_income))

income_group  Доход выше среднего  Доход ниже МРОТа  Доход ниже среднего  \
debt                                                                       
0                         15737.0               2.0                186.0   
1                          1439.0               NaN                 13.0   

income_group  Доход очень сильно выше среднего  Доход сильно выше среднего  
debt                                                                        
0                                        339.0                      3355.0  
1                                         22.0                       259.0  

Процент людей, не имеющих задолжности:
Для людей, чей доход выше среднего: 0.916
Для людей, чей доход ниже среднего: 0.935
Для людей, чей доход очень сильно выше среднего (в 10 и более раз): 0.939
Для людей, чей доход выше среднего (в 5-10 раз): 0.928


### Вывод

Вероятность того, что долг будет погашен у людей, чей доход превышает средний заработок в 10 и более раз - наиболе, - наиболее велика, что и неудивительно. Что странно, так это то, что у людей, с доходом ниже среднего вероятность выше, чем у людей с доходом выше среднего, но там и выборка ЗНАЧИТЕЛЬНО меньше.

- Как разные цели кредита влияют на его возврат в срок?

Поскольку у всех цели кредита можно разделить на: <b> автомобиль, свадьба, недвижимость и образование </b>. Выделим эти слова из лемм в столбце new_purpose и создадим ещё 1 слов, разделив всех заёмщиков на 4 категории.

In [927]:
def create_main_word(row):
    word = row['lem_purpose']
    if 'свадьба' in word:
        return 'свадьба'
    elif 'автомобиль' in word:
        return 'автомобиль'
    elif 'образование' in word:
        return 'образование'
    else:
        return 'недвижмость'

data['main_word_purpose'] = data.apply(create_main_word, axis =1)
purpose_pivot_table = data.groupby(['debt', 'main_word_purpose'])['gender'].aggregate('count').unstack()
purpose_pivot_table

main_word_purpose,автомобиль,недвижмость,образование,свадьба
debt,,,,
0,3884,9984,3625,2126
1,400,779,370,184


In [928]:
print(purpose_pivot_table)
for_1_purpose = purpose_pivot_table['автомобиль'][0] / purpose_pivot_table['автомобиль'].sum() 
for_2_purpose = purpose_pivot_table['недвижмость'][0] / purpose_pivot_table['недвижмость'].sum() 
for_3_purpose = purpose_pivot_table['образование'][0] / purpose_pivot_table['образование'].sum() 
for_4_purpose = purpose_pivot_table['свадьба'][0] / purpose_pivot_table['свадьба'].sum() 
print ()
print('Процент людей, не имеющих задолжности:')
print('Для людей, берущих кредит на автомобиль: {:.3f}'.format(for_1_purpose))
print('Для людей, берущих кредит на недвижимость: {:.3f}'.format(for_2_purpose))
print('Для людей, берущих кредит на образование: {:.3f}'.format(for_3_purpose))
print('Для людей, берущих кредит на свадьбу: {:.3f}'.format(for_4_purpose))

main_word_purpose  автомобиль  недвижмость  образование  свадьба
debt                                                            
0                        3884         9984         3625     2126
1                         400          779          370      184

Процент людей, не имеющих задолжности:
Для людей, берущих кредит на автомобиль: 0.907
Для людей, берущих кредит на недвижимость: 0.928
Для людей, берущих кредит на образование: 0.907
Для людей, берущих кредит на свадьбу: 0.920


### Вывод

Вероятность того, что кредит на недвижимость будет погашен больше чем вероятность того, что будет погашен кредит на свадьбу. Кредит, выданный на автомобили и образование  возвращается в срок в 0,907 случаев и это наименьший показатель из всех 4-х.

### Шаг 4. Общий вывод

Мы скачали данные с помощью библиотеки пандас, они оказались очень лохматыми, но мы их причесали: заполнили пропуски, удалили непонятные значения, избавились от дубликатов и перевели в целочисленный вид те столбцы. где это было необходимо. Затем с помощью лемматизации мы вывели ключевые слова из целей кредитования, что понадобилось нам в дальнейшем для определения зависимости между целью кредита и его возвратом в срок. Также мы использовали Категоризацию для построения сводных таблиц и вычисления по ним зависимостей. Результаты этих зависимостей можно посмотреть выше.